# Imports

In [ ]:
from experiments.inputSweep import config
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
from copy import deepcopy
from scipy.stats import median_abs_deviation, chi2
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LinearSegmentedColormap
from tqdm.notebook import tqdm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller, acf, q_stat
from scipy import stats
from sklearn.neighbors import KernelDensity
from statsmodels.nonparametric.kde import KDEUnivariate
from scipy.stats import friedmanchisquare
from statsmodels.stats.multitest import multipletests
np.random.seed(1)

# Load Data

In [ ]:
expSets = [config.experiments[i] for i in range(1,35)]

save_path = ...
modulation = "frequency" # "frequency"
if modulation == "amplitude":
    Settings = ["amplitude"]
    n_samples = 81
    n_tries = 110
elif modulation == "frequency":
    Settings = ["frequency"]
    n_samples = 49
    n_tries = 55
expSetsAmp = []
for expSet in expSets:
    for Setting in Settings:
        if Setting in expSet[4]:
            expSetAmp = deepcopy(expSet)
            expSetAmp[4] = [Setting]
            expSetsAmp.append(expSetAmp)

n_subsets = 9
names = [
    "2_0_",
    "2_1_",
    "3_0_",
    "3_1_",
    "3_2_",
    "4_0_",
    "4_1_",
    "4_2_",
    "4_3_",
]
            
window_list = np.arange(20, 201, 20)
number_of_bins = 8
totalNrNetworks = sum([len(e[0]) for e in expSetsAmp]) 
capacities = np.zeros((totalNrNetworks,4,len(window_list),number_of_bins))
capacities_b_small = np.zeros((totalNrNetworks,4,len(window_list),number_of_bins))
probabilities = np.zeros((totalNrNetworks,4,len(window_list),number_of_bins,n_samples))
encoding_values = np.zeros((totalNrNetworks,4,len(window_list),n_samples,n_tries)) 
n_bins = 250 * (np.arange(1, 7+1))
probabilities_y = np.zeros((totalNrNetworks,4,len(window_list),n_samples,250)) 
capacities_est = np.zeros((totalNrNetworks,4,n_subsets,number_of_bins)) 
capacities_est_b_small = np.zeros((totalNrNetworks,4,n_subsets,number_of_bins)) 
probability_y_est = np.zeros((totalNrNetworks,4,n_subsets,n_samples,250)) 
probability_est = np.zeros((totalNrNetworks,4,n_subsets,number_of_bins,n_samples)) 
p_values = np.zeros((totalNrNetworks,4,len(window_list),n_samples)) 
tot_len = 0
for j, expSet in enumerate(expSetsAmp): 
    for m,network in enumerate(expSet[0]): 
        filepath = os.path.join(save_path, f"processedAdv/{expSet[1]}_{expSet[4][0]}_{network}{expSet[-2]}.h5") 
        with h5py.File(filepath, 'r') as h5file: 
            print(filepath)
            for i, w in enumerate(window_list): 
                probabilities[tot_len+m,0,i,] = h5file[f"Capacity_Estimation/Rate/w{w}_p"][:]
                probabilities[tot_len+m,3,i,] = h5file[f"Capacity_Estimation/Latency/w{w}_p"][:]
                probabilities[tot_len+m,1,i,] = h5file[f"Capacity_Estimation/ISI/w{w}_p"][:]
                probabilities[tot_len+m,2, i,] = h5file[f"Capacity_Estimation/Phase/w{w}_p"][:]
                probabilities_y[tot_len+m,0,i,] = h5file[f"Capacity_Estimation/Rate/w{w}p_y"][:]
                probabilities_y[tot_len+m,3,i,] = h5file[f"Capacity_Estimation/Latency/w{w}p_y"][:]
                probabilities_y[tot_len+m,1,i,] = h5file[f"Capacity_Estimation/ISI/w{w}p_y"][:]
                probabilities_y[tot_len+m,2, i,] = h5file[f"Capacity_Estimation/Phase/w{w}p_y"][:]
                p_values[tot_len+m,0,i,] = h5file[f"AIS/Rate/w{w}_p_ais"][:]
                p_values[tot_len+m,3,i,] = h5file[f"AIS/Latency/w{w}_p_ais"][:]
                p_values[tot_len+m,1,i,] = h5file[f"AIS/ISI/w{w}_p_ais"][:]
                p_values[tot_len+m,2, i,] = h5file[f"AIS/Phase/w{w}_p_ais"][:]
                encoding_values[tot_len+m,0,i,] = np.mean(h5file[f"Rate/w{w}"][:],axis=-1)
                encoding_values[tot_len+m,3,i,] = np.mean(h5file[f"Latency/w{w}"][:],axis=-1)
                encoding_values[tot_len+m,1,i,] = np.mean(h5file[f"ISI/w{w}"][:],axis=-1)
                encoding_values[tot_len+m,2, i,] = np.mean(h5file[f"Phase/w{w}"][:],axis=-1)
                capacities[tot_len+m,0,i,] = h5file[f"Capacity_Estimation/Rate/w{w}_c"][:]
                capacities[tot_len+m,3,i,] = h5file[f"Capacity_Estimation/Latency/w{w}_c"][:]
                capacities[tot_len+m,1,i,] = h5file[f"Capacity_Estimation/ISI/w{w}_c"][:]
                capacities[tot_len+m,2, i,] = h5file[f"Capacity_Estimation/Phase/w{w}_c"][:]
            for i in range(n_subsets):
                probability_est[tot_len+m,0,i,] = h5file[f"Capacity_Estimation_Small/Rate/{names[i]}p"][:]
                probability_est[tot_len+m,3,i,] = h5file[f"Capacity_Estimation_Small/Latency/{names[i]}p"][:]
                probability_est[tot_len+m,1,i,] = h5file[f"Capacity_Estimation_Small/ISI/{names[i]}p"][:]
                probability_est[tot_len+m,2,i,] = h5file[f"Capacity_Estimation_Small/Phase/{names[i]}p"][:]
                probability_y_est[tot_len+m,0,i,] = h5file[f"Capacity_Estimation_Small/Rate/{names[i]}p_y"][:]
                probability_y_est[tot_len+m,3,i,] = h5file[f"Capacity_Estimation_Small/Latency/{names[i]}p_y"][:]
                probability_y_est[tot_len+m,1,i,] = h5file[f"Capacity_Estimation_Small/ISI/{names[i]}p_y"][:]
                probability_y_est[tot_len+m,2,i,] = h5file[f"Capacity_Estimation_Small/Phase/{names[i]}p_y"][:]
                capacities_est[tot_len+m,0,i,] = h5file[f"Capacity_Estimation_Small/Rate/{names[i]}c"][:]
                capacities_est[tot_len+m,3,i,] = h5file[f"Capacity_Estimation_Small/Latency/{names[i]}c"][:]
                capacities_est[tot_len+m,1,i,] = h5file[f"Capacity_Estimation_Small/ISI/{names[i]}c"][:]
                capacities_est[tot_len+m,2,i,] = h5file[f"Capacity_Estimation_Small/Phase/{names[i]}c"][:]
                capacities_est_b_small[tot_len+m,0,i,] = h5file[f"Capacity_Estimation_Small/Rate/{names[i]}c"][:]
                capacities_est_b_small[tot_len+m,3,i,] = h5file[f"Capacity_Estimation_Small/Latency/{names[i]}c"][:]
                capacities_est_b_small[tot_len+m,1,i,] = h5file[f"Capacity_Estimation_Small/ISI/{names[i]}c"][:]
                capacities_est_b_small[tot_len+m,2,i,] = h5file[f"Capacity_Estimation_Small/Phase/{names[i]}c"][:]
    tot_len += len(expSet[0])

# Plot

In [ ]:
bin_to_use = 6#0-7
# Step 1: Filter out trials where no capacity value exceeds 1
capacity_looked_at = capacities
valid_trials = np.any(capacity_looked_at[...,bin_to_use] > 1, axis=(1, 2))  # Check across all capacity types and windows
y_lim = (-0.25,2.6)

In [ ]:
# Filter capacities to include only valid trials
filtered_capacities = capacity_looked_at[valid_trials,...,bin_to_use] # -capacities_noise
#filtered_capacities = np.max(capacities[valid_trials],axis=-1) # -capacities_noise
#filtered_capacities -= np.mean(filtered_capacities,axis=(1,2))[:,None,None]
# Calculate the mean and standard deviation over the first dimension
mean_capacities = np.mean(filtered_capacities, axis=0)
#Standard Error of the Mean
std_capacities = np.std(filtered_capacities, axis=0) / np.sqrt(filtered_capacities.shape[0])

In [ ]:
# Convert window_list to milliseconds
window_list = np.arange(20, 201, 20)
window_ms = np.array(window_list) / 20
# Colors and labels for the four capacity types (use colorblind-friendly palette)
colors = ['#17becf', '#d95f02', '#7570b3', '#e7298a']
styles = ['solid','dotted','dashed','dashdot']
labels = ['Rate', 'ISI', 'Phase', 'TTFS']
symbols = ['D','s','o','v']
shifts = [-0.15,-0.05,0.05,0.15]

In [ ]:
# Set up the plot with standard figure size for a journal
fig, ax = plt.subplots(figsize=(8, 6))  # Adjust size to be more standard for a paper

# Set axis background to white (default)
ax.set_facecolor('white')
for i in range(4):
    # Line connecting the mean values
    ax.plot(window_ms+shifts[i], mean_capacities[i, :], label=labels[i], color=colors[i], linewidth=4)
    #for j,c in enumerate(filtered_capacities):
    #    ax.plot(window_ms, c[i, :], alpha=0.7, color=colors[j], linewidth=2, linestyle=styles[i])
for i in range(4):    
    # Error bars representing standard deviation at each point
    ax.scatter(window_ms+shifts[i], mean_capacities[i, :],marker=symbols[i], s=100, color=colors[i])
    ax.errorbar(window_ms+shifts[i], mean_capacities[i, :], yerr=std_capacities[i, :], 
                color=colors[i], capsize=3)

# Axis labels with appropriate fonts for a journal
plt.xlabel("Window Size [ms]", fontsize=14)
plt.ylabel("Capacity [bits / channel use]", fontsize=14)
plt.ylim(y_lim)
plt.xlim(0.5,10.5)
plt.legend(fontsize=12, loc='lower right')
# Ticks with standard font sizes
plt.tick_params(axis='both', labelsize=15)
plt.show()


In [ ]:
# Set up the new plot for each network's last entry values
fig, ax = plt.subplots(figsize=(8, 6))

# Loop over the 4 capacity types and plot their values across networks
sorted_indices = np.argsort(filtered_capacities[:,0,-1])
for i in range(4):
    # Scatter plot for the i-th capacity type
    ax.scatter(np.arange(len(filtered_capacities))+1, filtered_capacities[sorted_indices, i, -1], label=labels[i], 
               color=colors[i], marker=symbols[i], s=100)

# Axis labels
plt.xlabel("Network", fontsize=14)
plt.ylabel("Capacity [bits / channel use]", fontsize=14)
plt.ylim(y_lim)
# Ticks with standard font sizes
plt.tick_params(axis='both', labelsize=15)
ax.set_xticks(np.arange(len(filtered_capacities))+1)
# Adding a legend with standard font size
plt.legend(fontsize=12, loc='lower right')
ax.set_xticklabels([])
plt.show()

In [ ]:
# Set up the new plot for each network's last entry values
fig, ax = plt.subplots(figsize=(8, 6))

# Loop over the 4 capacity types and plot their values across networks
#for i in range(4):
#    # Scatter plot for the i-th capacity type
#    ax.scatter(np.arange(len(filtered_capacities))+1, filtered_capacities[:, i, -1], label=labels[i], 
#               color=colors[i], marker=symbols[i], s=100)

cdict = {
    'red':   [(0.0, 1.0, 1.0),  # Yellow
              (0.5, 0.5, 0.5),  # Transition to purple
              (1.0, 0.5, 0.5)], # Purple
    'green': [(0.0, 1.0, 1.0),
              (0.5, 0.0, 0.0),
              (1.0, 0.0, 0.0)],
    'blue':  [(0.0, 0.0, 0.0),
              (0.5, 0.5, 0.5),
              (1.0, 0.5, 0.5)]
}

#cmap = LinearSegmentedColormap('YellowPurple', segmentdata=cdict, N=256)
cmap = plt.get_cmap('summer')
sorted_indices = np.argsort(filtered_capacities[:,0,-1])[::-1]
noise = np.random.random(len(sorted_indices))*0.3-0.15
for j, index in enumerate(sorted_indices):
    # Scatter plot for the i-th capacity type# Loop over each network index
    plt.plot(np.arange(4)+noise[index],filtered_capacities[index, :, -1],c=cmap(j/len(filtered_capacities)),zorder=1)
for j, index in enumerate(sorted_indices):
    for i in range(4):
        ax.scatter(i+noise[index], filtered_capacities[index, i, -1], 
                   color=colors[i], marker=symbols[i], s=100,zorder=2)

# Axis labels
#plt.xlabel("Network Index", fontsize=14)
plt.ylabel("Capacity [bits / channel use]", fontsize=14)
plt.ylim(y_lim)
plt.xlim(-0.5,3.5)
# Ticks with standard font sizes # Adjust 0.15 based on your offset calculation
# Set x-axis ticks at these positions
# Axis labels with appropriate fonts for a journal
plt.xlabel("Encoding", fontsize=14)

# Ticks with standard font sizes
plt.tick_params(axis='both', labelsize=15)
ax.set_xticks(range(4))
ax.set_xticklabels(labels, fontsize=15)
# Adding a legend with standard font size
#plt.legend(fontsize=12, loc='lower right')

plt.show()

In [ ]:
w=-1
valid_trials = np.any(capacities[:,:,w] > 1, axis=(1, 2))
capacity_corrected = capacities[valid_trials]# - capacities_noise[valid_trials]
for k in range(capacity_corrected.shape[1]):
    # Calculate mean and standard error for error bars
    mean_values = np.mean(capacity_corrected[:, k, w], axis=0)
    error_values = np.std(capacity_corrected[:, k, w], axis=0) / np.sqrt(capacity_corrected.shape[0])
    
    # Plot with error bars
    plt.errorbar(
        x=np.arange(len(mean_values))+1,  # Assuming the x-axis is indices
        y=mean_values,
        yerr=error_values,
        label=labels[k],
        color=colors[k],
        linewidth=2,
        capsize=5  # Add caps to the error bars
    )

plt.legend()
plt.xlabel("X-axis Label")  # Customize as needed
plt.ylabel("Y-axis Label")  # Customize as needed
plt.title("Plot with Error Bars")
plt.show()

# AIS Statistical Analysis

In [ ]:
# Step 1: Combine within Durations for Each Experiment and Sample
for k in range(4): 
    for w in range(10):
        data = p_values[:, k, w, :]
        weights = np.zeros_like(data)+1
        weights[encoding_values[:,0, w].mean(axis=-1)>0.1] = 10#weights + np.exp()
        #for sample in range(n_biological_samples):
        #    print(combine_pvalues(np.clip(data[sample].flatten(), 1e-300, 1-1e-15),method="stouffer",weights=weights[sample].flatten())[1])
            #print(np.count_nonzero(data[sample]<0.05)/len(data[sample].flatten()),"%")
        # Step 3: Combine Across All Biological Samples for Global P-Value
        global_combined_p = combine_pvalues(np.clip(data.flatten(), 1e-300, 1-1e-15),method="stouffer",weights=weights.flatten())[1]
        #print(np.count_nonzero(data<0.05)/len(data.flatten()),"%")
        print("Global Combined p-value:", global_combined_p)

# ADF Test

In [ ]:
# Step 1: Combine within Durations for Each Experiment and Sample
combined_p_stouffer = np.zeros(4)
combined_p_fisher = np.zeros(4)
n_biological_samples = encoding_values.shape[0]
n_time_windows = encoding_values.shape[2]
n_experiments = encoding_values.shape[3]
adf_pvals = np.empty((4,n_biological_samples, n_time_windows,n_experiments))
pbar = tqdm(total=4*n_biological_samples*n_time_windows*n_experiments)
for k in range(4): 
    data = encoding_values[:, k, :, :]
    for sample_idx in range(n_biological_samples):
        for window_idx in range(n_time_windows):
            for exp_idx in range(n_experiments):
                if not data[sample_idx, window_idx, exp_idx].min()==data[sample_idx, window_idx, exp_idx].max():
                    series = data[sample_idx, window_idx, exp_idx]
                    _, p_value, _, _, _, _ = adfuller(series)
                    if not np.isnan(p_value):
                        adf_pvals[k,sample_idx, window_idx,exp_idx] =  p_value
                pbar.update(1)

In [ ]:
# Step 1: Combine within Durations for Each Experiment and Sample
for k in range(4): 
    for w in range(10):
        data = adf_pvals[k, :, w, :]
        weights = np.zeros_like(data)+1
        weights[encoding_values[:,0, w].mean(axis=-1)>0.1] = 10#weights + np.exp()
        global_combined_p = combine_pvalues(np.clip(data.flatten(), 1e-300, 1-1e-15),method="stouffer",weights=weights.flatten())[1]
        print("Global Combined p-value:", global_combined_p)

# Wilcoxon Test

In [ ]:
from scipy.stats import wilcoxon
from itertools import combinations
from statsmodels.stats.multitest import multipletests
# Simulate your data (replace with your actual array)
# Shape: (n_exp, 4, n_time)

n_exp, n_encodings, n_time = filtered_capacities.shape

# Preallocate matrices for results
pairwise_comparisons = list(combinations(range(n_encodings), 2))
significance_matrix = np.zeros((len(pairwise_comparisons), n_time), dtype=int)
p_value_matrix = np.full((len(pairwise_comparisons), n_time), np.nan)

for t in range(n_time):
    time_data = filtered_capacities[:, :, t]  # Data for all encodings at time t
    pairwise_pvals = []
    encoding_pairs = list(combinations(range(n_encodings), 2))  # All encoding pairs
    
    # Perform pairwise Wilcoxon signed-rank tests
    for enc1, enc2 in encoding_pairs:
        try:
            _, p_value = wilcoxon(time_data[:, enc1], time_data[:, enc2])
        except ValueError:
            p_value = np.nan  # Handle edge cases with insufficient variation
        pairwise_pvals.append(p_value)
    
    # Correct for multiple comparisons (FDR correction)
    corrected_pvals = multipletests(pairwise_pvals, method='fdr_bh')[1]
    
    # Populate matrices
    for i, ((enc1, enc2), corr_p_val) in enumerate(zip(encoding_pairs, corrected_pvals)):
        p_value_matrix[i, t] = corr_p_val
        significance_matrix[i, t] = int(corr_p_val < 0.05)

In [ ]:
# Convert window_list to milliseconds
time_labels = [f"{int(w / 20)} ms" for w in window_list]

# Construct LaTeX table header
header = (
    r"\begin{table}[ht]" + "\n"
    + r"\centering" + "\n"
    + r"\small" + "\n"  # Set table text to small size
    + r"\begin{tabular}{l" + "c" * n_time + r"}" + "\n"
    + r"\hline" + "\n"
    + "Pairwise Comparison"
    + "".join([f" & {t}" for t in time_labels])
    + r" \\" + "\n"
    + r"\hline" + "\n"
)

# Generate LaTeX table rows
latex_rows = []
for i, (enc1, enc2) in enumerate(pairwise_comparisons):
    row = f"{labels[enc1]} vs {labels[enc2]}"
    for t in range(n_time):
        # Bold significant values, round to 3 decimal places
        p_val = f"{p_value_matrix[i][t]:.3f}"
        if significance_matrix[i][t] == 1:
            row += f" & \\textbf{{{p_val}}}"
        else:
            row += f" & {p_val}"
    row += r" \\"  # Properly escape LaTeX newline
    latex_rows.append(row)

# Combine rows into the body
body = "\n".join(latex_rows) + "\n" + r"\hline" + "\n"

# Footer of the LaTeX table
footer = (
    r"\end{tabular}" + "\n"
    + r"\caption{Pairwise comparisons of encodings across time points. Significant p-values are bolded.}" + "\n"
    + r"\label{tab:pairwise_comparisons}" + "\n"
    + r"\end{table}"
)

# Combine all parts
latex_table = header + body + footer

# Output properly formatted LaTeX
from IPython.display import display, Markdown
display(Markdown(f"```latex\n{latex_table}\n```"))

# Capacity Estimation Bias

In [ ]:
#valid_trials = np.any(capacities_est > 1, axis=(1, 2, 3))  # Check across all capacity types and windows
capacities_est_corrected = capacities_est[valid_trials]#-capacities_est_noise
#capacities_est_corrected = (capacities_est)#[valid_trials]#-capacities_est_noise
capacities_1 = capacities[valid_trials,:,-1]
capacities_2 = np.mean(capacities_est_corrected[:,:,[0,1]],axis=-2)
capacities_3 = np.mean(capacities_est_corrected[:,:,[2,3,4]],axis=-2)
capacities_4 = np.mean(capacities_est_corrected[:,:,[5,6,7,8]],axis=-2)
capacities_check = np.stack((capacities_1,capacities_2,capacities_3,capacities_4))

In [ ]:
for k in range(capacities_check.shape[2]):
    data = capacities_check[:,:,k,bin_to_use]
    #data = np.diff(data,axis=0)
    plt.plot(np.arange(4),np.mean(data,axis=1),colors[k],linewidth=5)
    plt.plot(np.arange(4),data,colors[k],linestyle="dotted")
plt.title(f"{2**(bin_to_use+1)}")
plt.show()

In [ ]:
data = np.abs(np.mean(np.diff(capacities_check,axis=0),axis=0))
data_mean = np.mean(data,axis=0)
data_std = np.std(data,axis=0)
for k in range(capacities_check.shape[2]):
    #plt.plot(np.arange(1,data_mean[k].shape[-1])+1,data[:,k,1:].T,color=colors[k])
    plt.errorbar(x=np.arange(1,data_mean[k].shape[-1])+1+shifts[k], y=data_mean[k,1:],yerr=data_std[k,1:],label=labels[k],color=colors[k],linewidth=2,capsize=5)
    #plt.legend()
    #plt.plot(np.arange(3),data,colors[k],linestyle="dotted")
    #plt.title(f"{2**(b+1)}")
plt.errorbar(x=np.arange(1,data_mean[k].shape[-1])+1, y=np.mean(data_mean[:,1:],axis=0),yerr=np.mean(data_std[:,1:],axis=0),label="Mean",color="black",linewidth=2,capsize=5)
plt.ylim(0,0.14)
# Axis labels with appropriate fonts for a journal
plt.xticks(np.arange(1,data_mean[k].shape[-1])+1, [f"$2^{{{int(b)}}}$" for b in np.arange(1,data_mean[k].shape[-1])+1])
plt.xlabel("# Bins", fontsize=12)
plt.ylabel("Absolute Difference [Bits / Channel Use]", fontsize=12)
# Ticks with standard font sizes
plt.tick_params(axis='both', labelsize=13)
#plt.legend()
#plt.savefig(f"/itet-stor/kjoel/neuronies/biohybrid_cmos/zzz_Joel_copycat_pfft_howlong_can_we_make_folder_names/bias{Settings}.pdf", transparent=True, bbox_inches='tight')
plt.show()